In [1]:
VER = 1
import pandas as pd, numpy as np
import pickle, glob, gc

In [2]:
valid = pd.read_parquet('test_LB.pqt')
valid.head()

,session,aid,ts,type,d
0,12899779,59625,1661724000,0,0
1,12899780,1142000,1661724000,0,0
2,12899780,582732,1661724058,0,0
3,12899780,973453,1661724109,0,0
4,12899780,736515,1661724136,0,0


In [3]:
MN = valid.ts.min()
MN

1661724000

In [4]:
(MN - 1661119200) / (60*60*24)

7.0

In [5]:
#MN = 1661119200
valid['day'] = (valid.ts - MN) // (60*60*24)

In [6]:
valid.head()

,session,aid,ts,type,d,day
0,12899779,59625,1661724000,0,0,0
1,12899780,1142000,1661724000,0,0,0
2,12899780,582732,1661724058,0,0,0
3,12899780,973453,1661724109,0,0,0
4,12899780,736515,1661724136,0,0,0


In [7]:
valid = valid.sort_values(['session','ts'])

In [8]:
valid['x'] = valid.groupby(['session','d']).d.cumcount()

In [9]:
valid['a'] = 0
valid.loc[valid.x==0,'a'] = 1

In [10]:
valid = valid.sort_values(['session','ts'],ascending=[True,False])

In [11]:
valid['x'] = valid.groupby(['session','d']).d.cumcount()

In [12]:
valid['b'] = 0
valid.loc[valid.x==0,'b'] = 1

In [13]:
valid = valid.sort_values(['session','ts'])

In [14]:
valid = valid.drop('x',axis=1)

In [15]:
valid.head()

,session,aid,ts,type,d,day,a,b
0,12899779,59625,1661724000,0,0,0,1,1
1,12899780,1142000,1661724000,0,0,0,1,0
2,12899780,582732,1661724058,0,0,0,0,0
3,12899780,973453,1661724109,0,0,0,0,0
4,12899780,736515,1661724136,0,0,0,0,0


In [16]:
%%time
temp = [group for name, group in valid.sort_values(["session", "ts"]).groupby(["session"])]
print(len(temp))
# 1801251

1671803
CPU times: user 36.4 s, sys: 1.73 s, total: 38.2 s
Wall time: 38.2 s


In [17]:
%%time
PIECES = 10

for PART in range(PIECES):
    print(PART)
    
    print('### PART {} - from {} to {}'.format(PART + 1, PART * len(temp)//PIECES, (PART+1) * len(temp) // PIECES))
    
    mylist = [[h.session.iloc[0], h.aid.to_list(), h.type.to_list(), h.ts.to_list(), h.d.to_list(), 
               h.a.to_list(), h.b.to_list(), h.day.to_list() ] 
              for h in temp[PART * len(temp)//PIECES:(PART+1) * len(temp) // PIECES]]
    
    with open(f'../../data/infer_data/lists/test_group_tolist_{PART}_{VER}.pkl', 'wb') as f:
        pickle.dump(mylist, f) 

0
### PART 1 - from 0 to 167180
1
### PART 2 - from 167180 to 334360
2
### PART 3 - from 334360 to 501540
3
### PART 4 - from 501540 to 668721
4
### PART 5 - from 668721 to 835901
5
### PART 6 - from 835901 to 1003081
6
### PART 7 - from 1003081 to 1170262
7
### PART 8 - from 1170262 to 1337442
8
### PART 9 - from 1337442 to 1504622
9
### PART 10 - from 1504622 to 1671803
CPU times: user 13min 10s, sys: 10.8 s, total: 13min 21s
Wall time: 13min 20s
